# Clean Data

### Imports

In [30]:
import numpy as np
import pandas as pd


## Clean NFL Combine Data

In [31]:
combine_file = r'data\nfl_combine_1987_2020.csv'

df_raw_combine = pd.read_csv(combine_file)

df_raw_combine.head()

draft_file = r'data\espn_draft_history_2000_2021_cleaned.csv'
df_raw_draft = pd.read_csv(draft_file)

df_draft = df_raw_draft

df_draft.columns = df_draft.columns.str.lower()
df_draft.columns

Index(['year', 'round', 'pk(ovr)', 'team', 'name', 'position', 'school'], dtype='object')

In [32]:
combine_cols_to_drop = ['Unnamed: 0', 'Wonderlic']
df_combine = df_raw_combine
df_combine.drop(columns=combine_cols_to_drop, inplace=True)
df_combine.columns = df_combine.columns.str.lower()
df_combine.rename(columns={'college':'school'}, inplace=True)
df_combine.head()

,year,name,school,pos,height (in),weight (lbs),hand size (in),arm length (in),40 yard,bench press,vert leap (in),broad jump (in),shuttle,3cone,60yd shuttle
0,1987,Mike Adams,Arizona State,CB,69.8,198,8.50,30.50,4.42,13.0,32.0,118.0,4.60,NaN,11.91
1,1987,John Adickes,Baylor,C,74.8,266,10.25,30.00,4.97,25.0,26.5,103.0,4.60,NaN,NaN
2,1987,Tommy Agee,Auburn,FB,71.8,217,9.00,30.75,NaN,15.0,NaN,NaN,NaN,NaN,NaN
3,1987,David Alexander,Tulsa (OK),C,75.0,279,10.50,32.75,5.13,22.0,27.5,105.0,4.33,NaN,NaN
4,1987,Lyneal Alston,Southern Mississippi,WR,72.1,202,10.00,33.00,4.64,7.0,32.0,114.0,4.52,NaN,11.85


In [33]:
# combine results by position
df_combine['pos'].value_counts()

WR     1684
CB     1286
RB     1123
DE      974
OT      970
OLB     959
DT      951
OG      774
TE      729
QB      711
ILB     506
FS      495
SS      439
C       402
FB      303
P       157
K       139
LB       91
S        53
LS       22
EDG      22
OL       10
DL        4
DB        3
NT        1
Name: pos, dtype: int64

In [34]:
# Are there duplicated names?
df_combine['name'].value_counts(sort='descending').head(10)


Steve Smith         6
Chris Jones         6
Brandon Williams    5
Tony Jones          5
Chris Brown         5
Mike Davis          4
Brian Williams      4
Steve Williams      4
Kevin Williams      4
Anthony Johnson     4
Name: name, dtype: int64

In [50]:
# Do college names match in both datasets?
draft_school = pd.DataFrame(df_draft['school'].unique()).rename(columns={0:'school'})
draft_school['source'] = 'draft'
combine_school = pd.DataFrame(df_combine['school'].unique()).rename(columns={0:'school'})
combine_school['source'] = 'combine'
print(type(combine_school))
print(combine_school.head())
schools = draft_school.merge(combine_school, on='school', how='outer').sort_values(by='school')

schools.head(20)

# pd.concat([df_combine['school'], df_draft['school']], join='outer')

#df_combine.merge(df_draft, how='left', on='school')




<class 'pandas.core.frame.DataFrame'>
                 school   source
0         Arizona State  combine
1                Baylor  combine
2                Auburn  combine
3            Tulsa (OK)  combine
4  Southern Mississippi  combine


,school,source_x,source_y
202,Abilene Christian,draft,NaN
493,Abilene Christian (TX),NaN,combine
533,Adams State (CO),NaN,combine
297,Air Force,draft,combine
113,Akron,draft,combine
1,Alabama,draft,combine
166,Alabama A&M,draft,combine
203,Alabama State,draft,combine
454,Alabama-Birmingham,NaN,combine
200,Albany,draft,NaN


In [36]:
#df_combine.merge(df_draft, how='inner', on='school')